In [1]:
import numpy as np
import torch
import sys
import pickle
import os
from torchvision import models
from mftma.manifold_analysis_correlation import manifold_analysis_corr
from mftma.utils.make_manifold_data import make_manifold_data
from mftma.utils.activation_extractor import extractor
from mftma.utils.analyze_pytorch import analyze
import getpass
import argparse
from neural_manifold_utils import CFAR100_fake_dataset_mftma , save_dict, sub_data
from datetime import datetime
print('__cuda available ',torch.cuda.is_available())
print('__Python VERSION:', sys.version)
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
#import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms, models
import copy 

__cuda available  False
__Python VERSION: 3.7.7 (default, May  6 2020, 04:59:01) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
__CUDNN VERSION: None
__Number CUDA Devices: 0


In [3]:
user=getpass.getuser()
print(user)
if user=='eghbalhosseini':
    save_dir='/Users/eghbalhosseini/MyData/neural_manifolds/network_training_on_synthetic/'
    data_dir='/Users/eghbalhosseini/MyData/neural_manifolds/synthetic_datasets/'
    
elif user=='ehoseini':
    save_dir='/om/user/ehoseini/MyData/neural_manifolds/network_training_on_synthetic/'
    data_dir='/om/user/ehoseini/MyData/neural_manifolds/synthetic_datasets/'
train_dir=os.path.join(save_dir,'train_VGG16_synthdata_tree_nclass_50_n_exm_1000')
epoch_id=1
layer_num=0

eghbalhosseini


In [9]:
datafile=os.path.join('/Users/eghbalhosseini/Desktop/','train_epoch_1')
epoch_dat=pickle.load(open(datafile,'rb'))
#activations_cell=epoch_dat['activations_cell']

In [10]:
activations_cell = epoch_dat['activations_cell']
mfmta_data_ = {'mftma_results': [],
                   'train_spec': epoch_dat['train_spec'],
                   'train_accuracy': epoch_dat['train_accuracy'],
                   'train_success': epoch_dat['train_success'],
                   'epoch': epoch_dat['epoch'],
                   'layer_num':layer_num
                   }

In [18]:
((activations_cell[0])['layer_0_Input'][1]).shape

(100, 3, 32, 32)

In [19]:
hier_layer_names=[list(activations.keys()) for activations in activations_cell]
layer_n=[x[layer_num] for x in hier_layer_names]


In [20]:
layer_n

['layer_0_Input', 'layer_0_Input', 'layer_0_Input']

In [21]:
hier_layer_names = [list(activations.keys()) for activations in activations_cell]
layer_id = [x[layer_num] for x in hier_layer_names]
layer_activ_cell = [{layer_id[idx]: x[layer_id[idx]]} for idx, x in enumerate(activations_cell)]


In [26]:
((layer_activ_cell[0])['layer_0_Input'][1]).shape

(100, 3, 32, 32)

In [35]:
for hier_id, activ_hier in enumerate(layer_activ_cell):
    for layer, data in activ_hier.items():
        X = [d.reshape(d.shape[0], -1).T for d in data]
        N = X[0].shape[0]
        if N > 5000:
            print("Projecting {}".format(layer))
            M = np.random.randn(5000, N)
            M /= np.sqrt(np.sum(M * M, axis=1, keepdims=True))
            X = [np.matmul(M, d) for d in X]
        activ_hier[layer] = X
    layer_activ_cell[hier_id] = activ_hier

In [36]:
(layer_activ_cell[1]['layer_0_Input'][1]).shape

(3072, 100)

In [43]:
test_act={'layer_0_Input':layer_activ_cell[0]['layer_0_Input']}

In [44]:
for k, X, in test_act.items():
    # Analyze each layer's activations
    a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    # Compute the mean values
    a = 1 / np.mean(1 / a)
    r = np.mean(r)
    d = np.mean(d)
    print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))

    # Store for later
    capacities.append(a)
    radii.append(r)
    dimensions.append(d)
    correlations.append(r0)

layer_0_Input capacity: 0.101733, radius 0.753693, dimension 26.134985, correlation 0.042738


NameError: name 'capacities' is not defined

In [45]:
mftmas_cell=[]
for hier_id, activ_hier in enumerate(layer_activ_cell):
    data_ = {'capacities': [],
             'radii': [],
             'dimensions': [],
             'correlations':[],
             'layer':[],
            'n_hier_class':[],
            'hierarchy':hier_id}
    capacities = []
    radii = []
    dimensions = []
    correlations = []
    data_['layer']=layer_n[hier_id]
    data_['n_hier_class']=len(activ_hier[layer_n[hier_id]])
    for k, X, in activ_hier.items():
        print(k)
    # Analyze each layer's activations
        try:
            a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)

            # Compute the mean values
            a = 1/np.mean(1/a)
            r = np.mean(r)
            d = np.mean(d)
            print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))
        except:
            a=np.nan
            r=np.nan
            d=np.nan
            r0=np.nan
        # Store for later
        capacities.append(a)
        radii.append(r)
        dimensions.append(d)
        correlations.append(r0)
    # combine the results 
    data_['capacities']=capacities
    data_['radii']=radii
    data_['dimensions']=dimensions
    data_['correlations']=correlations
    mftmas_cell.append(data_)

layer_0_Input
layer_0_Input capacity: 0.102004, radius 0.754869, dimension 26.162407, correlation 0.042738
layer_0_Input
layer_0_Input capacity: 0.048093, radius 0.982784, dimension 41.949863, correlation 0.112095
layer_0_Input


In [ ]:
mftmas_cell

In [ ]:
len(activ_hier[layer_n[hier_id]])
#int(np.unique([len(activ_hier[x]) for x in data_['layers']]))

In [ ]:
for hier_id, activ_hier in enumerate(activations_cell):
    layer_names=list(activ_hier.keys())
    for layer, data, in activ_hier.items():
        X = [d.reshape(d.shape[0], -1).T for d in data]
        # Get the number of features in the flattened data
        N = X[0].shape[0]
        # If N is greater than 5000, do the random projection to 5000 features
        if N > 5000:
            print("Projecting {}".format(layer))
        #    M = np.random.randn(5000, N)
        #    M /= np.sqrt(np.sum(M*M, axis=1, keepdims=True))
        #    X = [np.matmul(M, d) for d in X]
        #activ_hier[layer] = X
    #activations_cell[hier_id]=activ_hier

In [ ]:
mftmas_cell=[]
for hier_id, activ_hier in enumerate(activations_cell):
    data_ = {'capacities': [],
             'radii': [],
             'dimensions': [],
             'correlations':[],
             'layers':[],
            'n_class':[],
            'hierarchy':hier_id}
    capacities = []
    radii = []
    dimensions = []
    correlations = []
    data_['layers']=activ_hier.keys()
    data_['hier_n_class']=int(np.unique([len(activ_hier[x]) for x in data_['layers']]))
    for k, X, in activ_hier.items():
    # Analyze each layer's activations
        a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    
        # Compute the mean values
        a = 1/np.mean(1/a)
        r = np.mean(r)
        d = np.mean(d)
        print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))
        # Store for later
        capacities.append(a)
        radii.append(r)
        dimensions.append(d)
        correlations.append(r0)
    # combine the results 
    data_['capacities']=capacities
    data_['radii']=radii
    data_['dimensions']=dimensions
    data_['correlations']=correlations

    mftmas_cell.append(data_)

    

In [ ]:
X[0].shape

In [ ]:
mfmta_data_ = {'mftma_results': mftmas_cell,
             'train_spec': epoch_dat['train_spec'],
             'train_accuracy': epoch_dat['train_accuracy'],
             'train_success': epoch_dat['train_success'],
             'epoch': epoch_dat['epoch']}


In [ ]:
[activations.keys() for activations in activations_cell]

In [ ]:
layer_names=list(activationenumerate))
layer=layer_names[1];
data=activations[layer]
X = [d.reshape(d.shape[0], -1).T for d in data]
N = X[0].shape[0]
# If N is greater than 5000, do the random projection to 5000 features
if N > 5000:
    print("Projecting {}".format(layer))
    M = np.random.randn(5000, N)
    M /= np.sqrt(np.sum(M * M, axis=1, keepdims=True))
    X = [np.matmul(M, d) for d in X]
data=X
capacities = []
radii = []
dimensions = []
correlations = []
a, r, d, r0, K = manifold_analysis_corr(data, 0, 300, n_reps=1)
a = 1 / np.mean(1 / a)
r = np.mean(r)
d = np.mean(d)
print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(layer, a, r, d, r0))


In [ ]:
result_save_path = save_dir + 'mftma_VGG16_synthdata_' + train_dataset.structure + '_nclass_' + str(
        int(train_dataset.n_class)) + '_n_exm_' + str(int(train_dataset.exm_per_class)) + str(1)

# First

In [4]:
datafile='synth_partition_nobj_50000_nclass_50_nfeat_3072_beta_0.01_sigma_1.50_norm_1.mat'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = CFAR100_fake_dataset_mftma(data_dir=os.path.join(data_dir, datafile))
sampled_classes = 50
examples_per_class = 100
data = make_manifold_data(dataset, sampled_classes, examples_per_class, seed=0)
model_save_path=save_dir+'VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))
model = models.vgg16(num_classes=dataset.n_class)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model = model.to(device)
model = model.eval()
activations = extractor(model, data, layer_types=['Conv2d', 'Linear'])
list(activations.keys())

<HDF5 dataset "data": shape (100000,), type "<f8"> is not a matlab type
<HDF5 dataset "ir": shape (100000,), type "<u8"> is not a matlab type
<HDF5 dataset "jc": shape (50051,), type "<u8"> is not a matlab type
data type not supported: graph, uint32


['layer_0_Input',
 'layer_1_Conv2d',
 'layer_3_Conv2d',
 'layer_6_Conv2d',
 'layer_8_Conv2d',
 'layer_11_Conv2d',
 'layer_13_Conv2d',
 'layer_15_Conv2d',
 'layer_18_Conv2d',
 'layer_20_Conv2d',
 'layer_22_Conv2d',
 'layer_25_Conv2d',
 'layer_27_Conv2d',
 'layer_29_Conv2d',
 'layer_33_Linear',
 'layer_36_Linear',
 'layer_39_Linear']

In [8]:
(activations['layer_0_Input'][1]).shape

(100, 3, 32, 32)

In [27]:
for layer, data, in activations.items():
    X = [d.reshape(d.shape[0], -1).T for d in data]
    # Get the number of features in the flattened data
    N = X[0].shape[0]
    # If N is greater than 5000, do the random projection to 5000 features
    if N > 5000:
        print("Projecting {}".format(layer))
        M = np.random.randn(5000, N)
        M /= np.sqrt(np.sum(M * M, axis=1, keepdims=True))
        X = [np.matmul(M, d) for d in X]
    activations[layer] = X

Projecting layer_1_Conv2d
Projecting layer_3_Conv2d
Projecting layer_6_Conv2d
Projecting layer_8_Conv2d
Projecting layer_11_Conv2d
Projecting layer_13_Conv2d
Projecting layer_15_Conv2d
Projecting layer_18_Conv2d
Projecting layer_20_Conv2d
Projecting layer_22_Conv2d


In [40]:
(activations['layer_0_Input'][1]).shape
test_act={'layer_0_Input':activations['layer_0_Input']}


dict

In [41]:
for k, X, in test_act.items():
    # Analyze each layer's activations
    a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    # Compute the mean values
    a = 1 / np.mean(1 / a)
    r = np.mean(r)
    d = np.mean(d)
    print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))

    # Store for later
    capacities.append(a)
    radii.append(r)
    dimensions.append(d)
    correlations.append(r0)

layer_0_Input capacity: 0.141250, radius 0.695877, dimension 20.883590, correlation 0.026355


NameError: name 'capacities' is not defined

In [ ]:
for k, X, in activations.items():
    # Analyze each layer's activations
    a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    # Compute the mean values
    a = 1 / np.mean(1 / a)
    r = np.mean(r)
    d = np.mean(d)
    print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))

    # Store for later
    capacities.append(a)
    radii.append(r)
    dimensions.append(d)
    correlations.append(r0)

In [ ]:
datafile='synth_partition_nobj_50000_nclass_50_nfeat_3072_beta_0.01_sigma_1.50_norm_1.mat'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = CFAR100_fake_dataset_mftma(data_dir=os.path.join(data_dir, datafile))
sampled_classes = 50
examples_per_class = 100
data = make_manifold_data(dataset, sampled_classes, examples_per_class, seed=0)
model_save_path=save_dir+'VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))
model = models.vgg16(num_classes=dataset.n_class)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model = model.to(device)
model = model.eval()
activations = extractor(model, data, layer_types=['Conv2d', 'Linear'])
list(activations.keys())
for layer, data, in activations.items():
    X = [d.reshape(d.shape[0], -1).T for d in data]
    # Get the number of features in the flattened data
    N = X[0].shape[0]
    # If N is greater than 5000, do the random projection to 5000 features
    if N > 5000:
        print("Projecting {}".format(layer))
        M = np.random.randn(5000, N)
        M /= np.sqrt(np.sum(M * M, axis=1, keepdims=True))
        X = [np.matmul(M, d) for d in X]
    activations[layer] = X
capacities = []
radii = []
dimensions = []
correlations = []
for k, X, in activations.items():
    # Analyze each layer's activations
    a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    # Compute the mean values
    a = 1 / np.mean(1 / a)
    r = np.mean(r)
    d = np.mean(d)
    print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))

    # Store for later
    capacities.append(a)
    radii.append(r)
    dimensions.append(d)
    correlations.append(r0)
names = list(activations.keys())
names = [n.split('_')[1] + ' ' + n.split('_')[2] for n in names]
# save the results:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
results_file = os.path.join(save_dir,'mftma_'+model_save_path+'_'+current_time)
data_1 = {'capacities': capacities,
             'radii': radii,
             'dimensions': dimensions,
             'correlations': correlations,
             'names': names,
             'analyze_exm_per_class': examples_per_class,
             'analyze_n_class': sampled_classes
             }

result_save_path=save_dir+'mftma_VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))+'_'+current_time
save_dict(data_1, result_save_path)


In [ ]:
capacities=data_1['capacities']
radii=data_1['radii']
dimensions=data_1['dimensions']
correlations=data_1['correlations']
names=data_1['names']

fig, axes = plt.subplots(1, 4, figsize=(18, 4))
plt.suptitle('Manifold Analysis results for Parition dataset; classes:'+ str(data_1['analyze_n_class'])+', example per class:'+str(data_1['analyze_exm_per_class']),
            fontsize=16,fontweight='bold')
axes[0].plot(capacities, linewidth=5)
axes[1].plot(radii, linewidth=5)
axes[2].plot(dimensions, linewidth=5)
axes[3].plot(correlations, linewidth=5)

axes[0].set_ylabel(r'$\alpha_M$', fontsize=18)
axes[1].set_ylabel(r'$R_M$', fontsize=18)
axes[2].set_ylabel(r'$D_M$', fontsize=18)
axes[3].set_ylabel(r'$\rho_{center}$', fontsize=18)

for ax in axes:
    ax.set_xticks([i for i, _ in enumerate(names)])
    ax.set_xticklabels(names, rotation=90, fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=14)

#plt.tight_layout()
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


# Second 

In [ ]:
datafile='synth_tree_nobj_50000_nclass_50_nfeat_3072_beta_0.01_sigma_1.50_norm_1.mat'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = CFAR100_fake_dataset_mftma(data_dir=os.path.join(data_dir, datafile))
sampled_classes = 50
examples_per_class = 100
data = make_manifold_data(dataset, sampled_classes, examples_per_class, seed=0)
model_save_path=save_dir+'VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))
model = models.vgg16(num_classes=dataset.n_class)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model = model.to(device)
model = model.eval()
activations = extractor(model, data, layer_types=['Conv2d', 'Linear'])
list(activations.keys())
for layer, data, in activations.items():
    X = [d.reshape(d.shape[0], -1).T for d in data]
    # Get the number of features in the flattened data
    N = X[0].shape[0]
    # If N is greater than 5000, do the random projection to 5000 features
    if N > 5000:
        print("Projecting {}".format(layer))
        M = np.random.randn(5000, N)
        M /= np.sqrt(np.sum(M * M, axis=1, keepdims=True))
        X = [np.matmul(M, d) for d in X]
    activations[layer] = X
capacities = []
radii = []
dimensions = []
correlations = []
for k, X, in activations.items():
    # Analyze each layer's activations
    a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    # Compute the mean values
    a = 1 / np.mean(1 / a)
    r = np.mean(r)
    d = np.mean(d)
    print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))

    # Store for later
    capacities.append(a)
    radii.append(r)
    dimensions.append(d)
    correlations.append(r0)
names = list(activations.keys())
names = [n.split('_')[1] + ' ' + n.split('_')[2] for n in names]
# save the results:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
results_file = os.path.join(save_dir,'mftma_'+model_save_path+'_'+current_time)
data_2 = {'capacities': capacities,
             'radii': radii,
             'dimensions': dimensions,
             'correlations': correlations,
             'names': names,
             'analyze_exm_per_class': examples_per_class,
             'analyze_n_class': sampled_classes
             }

result_save_path=save_dir+'mftma_VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))+'_'+current_time
save_dict(data_2, result_save_path)


In [ ]:
data=data_2
capacities=data['capacities']
radii=data['radii']
dimensions=data['dimensions']
correlations=data['correlations']
names=data['names']

fig, axes = plt.subplots(1, 4, figsize=(18, 4))
plt.suptitle('Manifold Analysis results for Tree dataset; classes:'+ str(data['analyze_n_class'])+', example per class:'+str(data['analyze_exm_per_class']),
            fontsize=16,fontweight='bold')
axes[0].plot(capacities, linewidth=5)
axes[1].plot(radii, linewidth=5)
axes[2].plot(dimensions, linewidth=5)
axes[3].plot(correlations, linewidth=5)

axes[0].set_ylabel(r'$\alpha_M$', fontsize=18)
axes[1].set_ylabel(r'$R_M$', fontsize=18)
axes[2].set_ylabel(r'$D_M$', fontsize=18)
axes[3].set_ylabel(r'$\rho_{center}$', fontsize=18)

for ax in axes:
    ax.set_xticks([i for i, _ in enumerate(names)])
    ax.set_xticklabels(names, rotation=90, fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=14)

#plt.tight_layout()
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Third

In [ ]:
datafile='synth_partition_nobj_100000_nclass_100_nfeat_3072_beta_0.01_sigma_1.50_norm_1.mat'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = CFAR100_fake_dataset_mftma(data_dir=os.path.join(data_dir, datafile))
sampled_classes = 100
examples_per_class = 200
data = make_manifold_data(dataset, sampled_classes, examples_per_class, seed=0)
model_save_path=save_dir+'VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))
model = models.vgg16(num_classes=dataset.n_class)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model = model.to(device)
model = model.eval()
activations = extractor(model, data, layer_types=['Conv2d', 'Linear'])
list(activations.keys())
for layer, data, in activations.items():
    X = [d.reshape(d.shape[0], -1).T for d in data]
    # Get the number of features in the flattened data
    N = X[0].shape[0]
    # If N is greater than 5000, do the random projection to 5000 features
    if N > 5000:
        print("Projecting {}".format(layer))
        M = np.random.randn(5000, N)
        M /= np.sqrt(np.sum(M * M, axis=1, keepdims=True))
        X = [np.matmul(M, d) for d in X]
    activations[layer] = X
capacities = []
radii = []
dimensions = []
correlations = []
for k, X, in activations.items():
    # Analyze each layer's activations
    a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    # Compute the mean values
    a = 1 / np.mean(1 / a)
    r = np.mean(r)
    d = np.mean(d)
    print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))

    # Store for later
    capacities.append(a)
    radii.append(r)
    dimensions.append(d)
    correlations.append(r0)
names = list(activations.keys())
names = [n.split('_')[1] + ' ' + n.split('_')[2] for n in names]
# save the results:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
results_file = os.path.join(save_dir,'mftma_'+model_save_path+'_'+current_time)
data_3 = {'capacities': capacities,
             'radii': radii,
             'dimensions': dimensions,
             'correlations': correlations,
             'names': names,
             'analyze_exm_per_class': examples_per_class,
             'analyze_n_class': sampled_classes
             }

result_save_path=save_dir+'mftma_VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))+'_'+current_time
save_dict(data_3, result_save_path)


In [ ]:
data=data_3
capacities=data['capacities']
radii=data['radii']
dimensions=data['dimensions']
correlations=data['correlations']
names=data['names']

fig, axes = plt.subplots(1, 4, figsize=(18, 4))
plt.suptitle('Manifold Analysis results for Parition dataset; classes:'+ str(data['analyze_n_class'])+', example per class:'+str(data['analyze_exm_per_class']),
            fontsize=16,fontweight='bold')
axes[0].plot(capacities, linewidth=5)
axes[1].plot(radii, linewidth=5)
axes[2].plot(dimensions, linewidth=5)
axes[3].plot(correlations, linewidth=5)

axes[0].set_ylabel(r'$\alpha_M$', fontsize=18)
axes[1].set_ylabel(r'$R_M$', fontsize=18)
axes[2].set_ylabel(r'$D_M$', fontsize=18)
axes[3].set_ylabel(r'$\rho_{center}$', fontsize=18)

for ax in axes:
    ax.set_xticks([i for i, _ in enumerate(names)])
    ax.set_xticklabels(names, rotation=90, fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=14)

#plt.tight_layout()
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# forth : super class analysis 

In [ ]:
datafile='synth_tree_nobj_50000_nclass_50_nfeat_3072_beta_0.01_sigma_1.50_norm_1.mat'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = CFAR100_fake_dataset_mftma(data_dir=os.path.join(data_dir, datafile))

In [ ]:
num_hierarchy=len(dataset.vals.hierarchical_class_ids)
hier_classes=[x-1 for x in dataset.vals.hierarchical_class_ids]
hier_n_class=[int(max(x)+1) for x in hier_classes]
exm_per_class=10

In [ ]:
hier_dat_mtmfa=[]
for idx, x in enumerate(hier_classes):
        dat_mfmta=[]
        dat_mfmta=copy.deepcopy(dataset)
        dat_mtmfa.target=hier_classes[idx]
        hier_dat_mtmfa.append(copy.deepcopy(dat_mtmfa));


In [ ]:
hier_dat_mtmfa[3].target

In [ ]:
make_manifold_data([dataset.dat,dataset.target], 50, exm_per_class, seed=0)

In [ ]:
hier_sample_mtmfa= [make_manifold_data(x, hier_n_class[idx], exm_per_class, seed=0) for idx, x in enumerate(hier_dat_mtmfa)]

In [ ]:
model_save_path=save_dir+'VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))
model = models.vgg16(num_classes=dataset.n_class)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model = model.to(device)
model = model.eval()

In [ ]:
activations_cell = [extractor(model,x,layer_types=['Conv2d', 'Linear']) for x in hier_sample_mtmfa]

In [ ]:
dataset.structure

In [ ]:
model_save_path=save_dir+'VGG16_synthdata_'+dataset.structure+'_nclass_'+str(int(dataset.n_class))+'_n_exm_'+str(int(dataset.exm_per_class))
model = models.vgg16(num_classes=dataset.n_class)
model.load_state_dict(torch.load(model_save_path, map_location=device))
model = model.to(device)
model = model.eval()
activations = extractor(model, data, layer_types=['Conv2d', 'Linear'])
list(activations.keys())
for layer, data, in activations.items():
    X = [d.reshape(d.shape[0], -1).T for d in data]
    # Get the number of features in the flattened data
    N = X[0].shape[0]
    # If N is greater than 5000, do the random projection to 5000 features
    if N > 5000:
        print("Projecting {}".format(layer))
        M = np.random.randn(5000, N)
        M /= np.sqrt(np.sum(M * M, axis=1, keepdims=True))
        X = [np.matmul(M, d) for d in X]
    activations[layer] = X
capacities = []
radii = []
dimensions = []
correlations = []
for k, X, in activations.items():
    # Analyze each layer's activations
    a, r, d, r0, K = manifold_analysis_corr(X, 0, 300, n_reps=1)
    # Compute the mean values
    a = 1 / np.mean(1 / a)
    r = np.mean(r)
    d = np.mean(d)
    print("{} capacity: {:4f}, radius {:4f}, dimension {:4f}, correlation {:4f}".format(k, a, r, d, r0))

    # Store for later
    capacities.append(a)
    radii.append(r)
    dimensions.append(d)
    correlations.append(r0)
names = list(activations.keys())
names = [n.split('_')[1] + ' ' + n.split('_')[2] for n in names]
# save the results:
current_time = datetime.now().strftime('%b%d_%H-%M-%S')
results_file = os.path.join(save_dir,'mftma_'+model_save_path+'_'+current_time)
data_4 = {'capacities': capacities,
             'radii': radii,
             'dimensions': dimensions,
             'correlations': correlations,
             'names': names,
             'analyze_exm_per_class': examples_per_class,
             'analyze_n_class': sampled_classes
             }

result_save_path=save_dir+'mftma_VGG16_synthdata_'+dataset.structure+'_nclass_'+str(hier_n_class)+'_n_exm_'+str(int(dataset.exm_per_class))+'_'+current_time
save_dict(data_2, result_save_path)


In [ ]:
data=data_4
capacities=data['capacities']
radii=data['radii']
dimensions=data['dimensions']
correlations=data['correlations']
names=data['names']

fig, axes = plt.subplots(1, 4, figsize=(18, 4))
plt.suptitle('Manifold Analysis results for Parition dataset; classes:'+ str(data['analyze_n_class'])+', example per class:'+str(data['analyze_exm_per_class']),
            fontsize=16,fontweight='bold')
axes[0].plot(capacities, linewidth=5)
axes[1].plot(radii, linewidth=5)
axes[2].plot(dimensions, linewidth=5)
axes[3].plot(correlations, linewidth=5)

axes[0].set_ylabel(r'$\alpha_M$', fontsize=18)
axes[1].set_ylabel(r'$R_M$', fontsize=18)
axes[2].set_ylabel(r'$D_M$', fontsize=18)
axes[3].set_ylabel(r'$\rho_{center}$', fontsize=18)

for ax in axes:
    ax.set_xticks([i for i, _ in enumerate(names)])
    ax.set_xticklabels(names, rotation=90, fontsize=16)
    ax.tick_params(axis='both', which='major', labelsize=14)

#plt.tight_layout()
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
data_4